In [ ]:
%load_ext tensorboard

import math
import gym 
import plotly.express as px
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import random
from collections import Counter, deque
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

import os
import glob
from IPython import display as ipythondisplay
from tqdm.notebook import tqdm
from gym.wrappers import Monitor
from IPython.display import HTML
import base64
import io
import pickle
import torch
from torch import nn
import kornia.augmentation as aug
warnings.filterwarnings("ignore")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
# display.start()

In [ ]:
import os, sys, copy, argparse, shutil
def parse_arguments():
    parser = argparse.ArgumentParser(description='Deep Q Network Argument Parser')
    parser.add_argument('--seed', dest='seed', type=int, default=1)
    parser.add_argument('--env', dest='env', type=str, default='CartPole-v0')
    parser.add_argument('--save_interval', type=int, default=50, help='save model every n episodes')
    parser.add_argument('--log_interval', type=int, default=10, help='logging every n episodes')
    parser.add_argument('--render', help='render', type=int, default=1)
    parser.add_argument('--batch_size', help='batch_size', type=int, default=32)
    parser.add_argument('--train_freq', help='train_frequency', type=int, default=1)
    parser.add_argument('--max_episode', help='maximum episode', type=int, default=None)
    parser.add_argument('--max_timesteps', help='maximum timestep', type=int, default=100000000)
    parser.add_argument('--lr', dest='lr', type=float, default=0.00025)
    parser.add_argument('--lr_decay', action='store_true', help='decay learning rate')
    parser.add_argument('--gamma', help='discount_factor', type=float, default=0.99)
    parser.add_argument('--warmup_mem', type=int, help='warmup memory size', default=1000)
    parser.add_argument('--frame_skip', type=int, help='number of frames to skip for each action', default=3)
    parser.add_argument('--frame_stack', type=int, help='number of frames to stack', default=4)
    parser.add_argument('--memory', help='memory size', type=int, default=1000000)
    parser.add_argument('--initial_epsilon', '-ie', help='initial_epsilon', type=float, default=0.5)
    parser.add_argument('--final_epsilon', '-fe', help='final_epsilon', type=float, default=0.05)
    parser.add_argument('--max_epsilon_decay_steps', '-eds', help='maximum steps to decay epsilon', type=int, default=100000)
    parser.add_argument('--max_grad_norm', type=float, default=None, help='maximum gradient norm')
    parser.add_argument('--soft_update', '-su', action='store_true', help='soft update target network')
    parser.add_argument('--double_q', '-dq', action='store_true', help='enabling double DQN')
    parser.add_argument('--dueling_net', '-dn', action='store_true', help='enabling dueling network')
    parser.add_argument('--test', action='store_true', help='test the trained model')
    parser.add_argument('--tau', type=float, default=0.01, help='tau for soft target network update')
    parser.add_argument('--hard_update_freq', '-huf', type=int, default=2000, help='hard target network update frequency')
    parser.add_argument('--save_dir', type=str, default='./data')
    parser.add_argument('--resume_step', '-rs', type=int, default=None)
    return parser.parse_args()

In [ ]:
#@title Set up constants for env and training
test = False 
save_dir = './data'
render = False
max_episode = None
max_timesteps = 100000000


In [ ]:
#@title Augmentations

color_jitter = aug.ColorJitter(
        brightness=np.random.random(),
        contrast=np.random.random(),
        saturation=np.random.random(),
        hue=np.random.random(),
        p=0.5
        )
random_elastic_transform = aug.RandomElasticTransform()
random_fisheye = aug.RandomFisheye(
        center_x=torch.tensor([-.3, .3]),
        center_y=torch.tensor([-.3, .3]),
        gamma=torch.tensor([.9, 1.]),
        )
random_color_equalize = aug.RandomEqualize()
random_gaussian_blur = aug.RandomGaussianBlur(
        kernel_size=(9, 9),
        sigma = (5., 5.)
        )
random_gaussian_noise = aug.RandomGaussianNoise()
random_horizontal_flip = aug.RandomHorizontalFlip()
random_color_invert = aug.RandomInvert()
random_perspective_shift = aug.RandomPerspective()
random_shift = nn.Sequential(aug.RandomCrop((190, 140)), nn.ReplicationPad2d(20), aug.RandomCrop((210, 160)))

In [ ]:
def tie_weights(src, trg):
    assert type(src) == type(trg)
    trg.weight = src.weight
    trg.bias = src.bias


# for 84 x 84 inputs
OUT_DIM = {2: 39, 4: 35, 6: 31}
# for 64 x 64 inputs
OUT_DIM_64 = {2: 29, 4: 25, 6: 21}

''' TODO change the layer parameters ''' 
class PixelEncoder(nn.Module):
    """Convolutional encoder of pixels observations."""
    # [210, 160] --> crop [190, 140]
    def __init__(self, obs_shape, feature_dim=50, num_layers=3, num_filters=64, output_logits=False):
        super().__init__()

        assert len(obs_shape) == 3
        self.obs_shape = obs_shape
        self.feature_dim = feature_dim
        self.num_layers = num_layers
        
        # 160, 210, 3 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=5)
        # Input to conv2: 32, 42, 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        # Input to conv3: 15, 20, 64
        self.conv3 = nn.Conv2d(64, 64, kernel_size=4, stride=1)
        # Output from conv3: 12, 17, 64

        # out_dim = OUT_DIM_64[num_layers] if obs_shape[-1] == 64 else OUT_DIM[num_layers]
        out_dims = (12, 17)
        self.fc = nn.Linear(num_filters * out_dims[0] * out_dims[1], self.feature_dim)
        self.ln = nn.LayerNorm(self.feature_dim)

        self.outputs = dict()
        self.output_logits = output_logits

    def reparameterize(self, mu, logstd):
        std = torch.exp(logstd)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward_conv(self, obs):
        obs = obs / 255.
        self.outputs['obs'] = obs
        conv1 = torch.relu(self.conv1(obs))
        self.outputs['conv1'] = conv1
        conv2 = torch.relu(self.conv2(conv1))
        self.outputs['conv2'] = conv2
        conv3 = torch.relu(self.conv3(conv2))
        self.outputs['conv3'] = conv3

        h = conv3.view(conv3.size(0), -1)
        return h

    def forward(self, obs, detach=False):
        h = self.forward_conv(obs)

        if detach:
            h = h.detach()

        h_fc = self.fc(h)
        self.outputs['fc'] = h_fc

        h_norm = self.ln(h_fc)
        self.outputs['ln'] = h_norm

        if self.output_logits:
            out = h_norm
        else:
            out = torch.tanh(h_norm)
            self.outputs['tanh'] = out

        return out

    def copy_conv_weights_from(self, source):
        """Tie convolutional layers"""
        # only tie conv layers
        for i in range(self.num_layers):
            tie_weights(src=source.convs[i], trg=self.convs[i])

    def log(self, L, step, log_freq):
        if step % log_freq != 0:
            return

        for k, v in self.outputs.items():
            L.log_histogram('train_encoder/%s_hist' % k, v, step)
            if len(v.shape) > 2:
                L.log_image('train_encoder/%s_img' % k, v[0], step)

        for i in range(self.num_layers):
            L.log_param('train_encoder/conv%s' % (i + 1), self.convs[i], step)
        L.log_param('train_encoder/fc', self.fc, step)
        L.log_param('train_encoder/ln', self.ln, step)

In [ ]:
def wrap_env(env, train=True):
    suffix = 'train' if train else 'test'
    monitor_dir = os.path.join(save_dir, 'monitor_%s' % suffix)
    os.makedirs(monitor_dir, exist_ok=True)
    if not train:
        video_save_interval = 10
        env = Monitor(env, directory=monitor_dir,
                      video_callable=lambda episode_id: episode_id % video_save_interval == 0,
                      force=True)
    else:
        if render:
            if max_episode is not None:
                video_save_interval = int(max_episode / 3)
            else:
                video_save_interval = int(max_timesteps / float(env._max_episode_steps) / 3)
            env = Monitor(env, directory=monitor_dir,
                          video_callable=lambda episode_id: episode_id % video_save_interval == 0,
                          force=True)
        else:
            env = Monitor(env, directory=monitor_dir, video_callable=False, force=True)
    return env

In [ ]:
class ReplayMemory(object):
    def __init__(self, max_epi_num=2000, max_epi_len=200):
        # capacity is the maximum number of steps in memory
        self.max_epi_num = max_epi_num
        self.max_epi_len = max_epi_len
        # saves each tuple of (state, action, next state, reward)
        self.capacity = 1000 # self.max_epi_num * max_epi_len
        self.idx = 0
        self.obs_memory = np.zeros((self.capacity, 210, 160, 3)) # deque(maxlen=self.max_epi_num * max_epi_len)
        self.next_memory = np.zeros((self.capacity, 210, 160, 3))
        self.act_memory = np.zeros((self.capacity, 1))
        self.reward_memory = np.zeros((self.capacity, 1))
        self.is_av = False
        self.current_epi = 0

    def reset(self):
        self.current_epi = 0
        self.memory.clear()

    ''' deprecated for tuple buffer '''
    def create_new_epi(self):
        pass

    def remember(self, state, next_state, action, reward):
        if self.idx == self.capacity:
            self.idx = 0
        self.obs_memory[self.idx] = state.copy()
        self.next_memory[self.idx] = next_state.copy()
        self.act_memory[self.idx] = action
        self.reward_memory[self.idx] = reward
        self.idx += 1
        
        '''
        if len(self.memory) < self.capacity:
            new_sample = np.array([state, action, reward, next_state])
            if len(self.memory) == 0:
                self.memory = [new_sample]
            else:
                length = len(self.memory)
                self.memory.append(new_sample)
        '''
                
    # samples batch_size
    def sample(self, batch_size):
        if batch_size < self.idx:
            idx = np.random.randint(0, self.idx - 1, batch_size)
            return self.obs_memory[idx], self.next_memory[idx], self.act_memory[idx], self.reward_memory[idx]
        return self.obs_memory, self.next_memory, self.act_memory, self.reward_memory

    def size(self):
        return self.idx

    def is_available(self):
        self.is_av = True
        if self.idx <= 1:
            self.is_av = False
        return self.is_av

    def print_info(self):
        pass

In [ ]:
#@title Create a training conv agent
import torch.nn.functional as F

class DQNetworkConv(nn.Module):
    def __init__(self, in_channels, act_dim, dueling=False):
        super(DQNetworkConv, self).__init__()
        self.act_dim = act_dim
        self.dueling = dueling
        # 160, 210, 3 
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=5, stride=5)
        # Input to conv2: 32, 42, 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        # Input to conv3: 15, 20, 64
        self.conv3 = nn.Conv2d(64, 64, kernel_size=4, stride=1)
        # Output from conv3: 12, 17, 64
        if self.dueling:
            self.v_fc4 = nn.Linear(12 * 17 * 64, 512)
            self.adv_fc4 = nn.Linear(12 * 17 * 64, 512)
            self.v_fc5 = nn.Linear(512, 1)
            self.adv_fc5 = nn.Linear(512, self.act_dim)
        else:
            self.fc4 = nn.Linear(12 * 17 * 64, 512)
            self.fc5 = nn.Linear(512, self.act_dim)
        self.parameters = (list(self.conv1.parameters())) + (list(self.conv2.parameters())) + (list(self.conv3.parameters())) + (list(self.fc4.parameters())) + (list(self.fc5.parameters()))

    def forward(self, st):
        out = F.relu(self.conv1(st))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out.view(out.size(0), -1)
        if self.dueling:
            val = F.relu(self.v_fc4(out))
            adv = F.relu(self.adv_fc4(out))
            val = self.v_fc5(val)
            adv = self.adv_fc5(adv)
            out = val.expand_as(adv) + adv - adv.mean(-1, keepdim=True).expand_as(adv)
        else:
            out = F.relu(self.fc4(out))
            out = self.fc5(out)
        return out

In [ ]:
#@title Create a training FC agent
import torch.nn.functional as F

class DQNetworkFC(nn.Module):
    def __init__(self, z_dim, act_dim, dueling=False):
        super(DQNetworkFC, self).__init__()
        self.act_dim = act_dim
        self.input_dim = z_dim 
        self.dueling = dueling
        if self.dueling:
            self.v_fc1 = nn.Linear(z_dim, 512)
            self.adv_fc1 = nn.Linear(z_dim, 512)
            self.v_fc2 = nn.Linear(512, 1)
            self.adv_fc2 = nn.Linear(512, 256)
            self.v_fc3 = nn.Linear(256, 1)
            self.adv_fc3 = nn.Linear(256, self.act_dim)
        else:
            self.fc1 = nn.Linear(z_dim, 512)
            self.fc2 = nn.Linear(512, 256)
            self.fc3 = nn.Linear(256, self.act_dim)

    def forward(self, st):
        out = F.relu(self.fc1(st))
        out = F.relu(self.fc2(out))
        ''' Do we need a relu on the last layer if the output is probability over action space? '''
        out = F.relu(self.fc3(out))
        return out

In [ ]:
# import torch.nn.functional as F

def process_obs(obs, pad=True):
    '''
    Pad to match the expected encoder input size
    '''
    obs = torch.Tensor(obs)
    if len(obs.shape) < 4:
        obs = obs.unsqueeze(0)
        
    if pad:
        w_pad = torch.zeros((obs.shape[0], (224-obs.shape[1])//2, obs.shape[2], 3))
        obs = torch.cat((w_pad, obs, w_pad), 1)
        h_pad = torch.zeros((obs.shape[0], 224, (224 - obs.shape[2])//2, 3))
        obs = torch.cat((h_pad, obs, h_pad), 2)
    obs = obs.permute(0, 3, 1, 2)
    return obs

In [ ]:
def take_action(env, action):
    state, rew, done, _ = env.step(action)
    obs = env.render(mode='rgb_array')
    return obs, rew, done

In [ ]:
MAX_STEPS = 200

In [ ]:
class CURL(nn.Module):
    """
    CURL
    """

    def __init__(self, obs_shape, z_dim, batch_size, encoder, output_type="continuous", critic=None, critic_target=None):
        super(CURL, self).__init__()
        self.batch_size = batch_size

        # self.encoder = critic.encoder
        self.encoder = encoder 

        # self.encoder_target = critic_target.encoder 

        self.W = nn.Parameter(torch.rand(z_dim, z_dim))
        self.output_type = output_type

    def encode(self, x, detach=False, ema=False):
        """
        Encoder: z_t = e(x_t)
        :param x: x_t, x y coordinates
        :return: z_t, value in r2
        """
        if ema:
            with torch.no_grad():
                z_out = self.encoder_target(x)
        else:
            z_out = self.encoder(x)

        if detach:
            z_out = z_out.detach()
        return z_out

    def compute_logits(self, z_a, z_mod):
        """
        Uses logits trick for CURL:
        - compute (B,B) matrix z_a (W z_pos.T)
        - positives are all diagonal elements
        - negatives are all other elements
        - to compute loss use multiclass cross entropy with identity matrix for labels
        """
        Wz = torch.matmul(self.W, z_mod.T)  # (z_dim,B)
        logits = torch.matmul(z_a, Wz)  # (B,B)
        logits = logits - torch.max(logits, 1)[0][:, None]
        return logits

In [ ]:
#@title Generate a batch of negatively labelled examples given observations

def generate_negatives(obs):
    neg_idx = np.random.randint(len(obs), size=len(obs))
    pos_idx = np.arange(len(obs))
    resample = (neg_idx == pos_idx)
    for (i, r) in enumerate(resample):
        if r:
            idx = neg_idx[i]
        else:
            idx = np.random.randint(0, len(obs), 1)[0]
            while idx == i:
                idx = np.random.randint(0, len(obs), 1)[0]
        neg_idx[i] = idx
    return (obs[neg_idx]).copy()


In [ ]:
#@title Create a training agent (wrapper for conv agent)
from torch.autograd import Variable

GAMMA = 0.99

class Agent(object):
    def __init__(self, act_dim, in_channels=3, max_epi_num=50, max_epi_len=300, encoder=None, conv_net=False, z_dim=50):
        self.N_action = act_dim
        self.max_epi_num = max_epi_num
        self.max_epi_len = max_epi_len
        ''' To decide when to copy weights to the target network '''
        self.num_param_updates = 0
        
        ''' Choose between a pre-trained (on Imagenet) encoder or CURL encoder'''
        # pretrained encoder --> get its hidden features as the state encoding for RL training
        self.encoder = encoder
        if conv_net:
            self.q_net = DQNetworkConv(in_channels, act_dim)
            self.target = DQNetworkConv(in_channels, act_dim)
        else:
            ''' if using the encoder head for contrastive loss '''
            self.q_net = DQNetworkFC(z_dim, act_dim)
            self.target = DQNetworkFC(z_dim, act_dim)
        self.buffer = ReplayMemory(max_epi_num=self.max_epi_num, max_epi_len=self.max_epi_len)
        self.gamma = 0.999
        self.loss_fn = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=1e-3)

    def remember(self, state, action, reward, next_state):
        self.buffer.remember(state, next_state, action, reward)

    ''' Copy the weights to the target network every 100 updates '''
    def train(self, batch_size=32, target_update_freq=100, use_encoder=True):
        if self.buffer.is_available():
            obs, next_obs, action_list, reward_list = self.buffer.sample(batch_size)
            
            ''' Pass through the encoder to get encodings
             If also training the contrastive loss
             include that here! 
             1. data augmentation to create pos and negative pairs
             2. encoder 
             3. update encoder loss function (using a separate optimizer) or add to the loss computed below
            '''

            losses = [] 

            ''' If not using the encoder, pass the obs directly to the CNN '''
            obs = process_obs(obs)
            next_obs = process_obs(next_obs)
            # estimate current q values from obs
            z = self.encoder(obs)
            next_z = self.encoder(next_obs).detach()
            
            Qs = self.q_net(z)
            
            ''' find target q values ''' 
            # find next max q values based on next observations
            next_qs = self.target(next_z).detach()
            next_Qs, max_actions = next_qs.max(1)[0], next_qs.max(1)[1]
            
            '''
            # create a mask for actions taken in the env
            mask = torch.zeros((Qs.shape))
            for (i, action) in enumerate(action_list):
                mask[i, int(action[0])] = 1.0
            mask = Variable(mask, requires_grad=True)
            # Qs = torch.gather(Qs, dim=1, index=torch.tensor(action_list, dtype=torch.int64)).reshape(-1)
            
            next_Qs = next_Qs.numpy()
            target_qs = torch.tensor(reward_list.squeeze(-1) + GAMMA * next_Qs).long()
            # try to set Qs equal to target_Qs 
            Qs = Qs * mask
            target_Qs = torch.zeros((Qs.shape))
            for (i, q) in enumerate(target_qs):
                target_Qs[i, int(action_list[i][0])] = q
            '''
            
            next_Qs = next_Qs.numpy()
            
            target_qs = torch.tensor(reward_list.squeeze(-1) + GAMMA * next_Qs).float()
            target_qs = target_qs.repeat(18).reshape(18, 32)
            target_qs = target_qs.permute(1, 0)
            # Qs.requires_grad = True
            q_loss = torch.sum((Qs - target_qs) ** 2)
            # q_loss = self.loss_fn(Qs, target_qs).long()
            losses.append(q_loss)
            
            ''' Loss update for q network and encoder head '''
            losses = torch.stack(losses).sum()
            self.optimizer.zero_grad()
            losses.backward()
            self.optimizer.step()

            self.num_param_updates += 1
            if self.num_param_updates % target_update_freq == 0:
                self.target.load_state_dict(self.q_net.state_dict())

    # TODO: check the sizes of inputs and outputs
    def get_action(self, obs, epsilon, use_encoding=True):
        ''' 
         If using an encoder, need to pass that thorugh the encoder
         then use the encoding to pass through self.conv_net
        '''
        obs = process_obs(obs)

        # Dividing obs by 255 is handled in encoder forward (only needed for use_encoding=False)
        if len(obs.shape) == 1:
            obs = obs.unsqueeze(0)

        # epsilon greedy for selecting which action to take
        if random.random() > epsilon:
            zs = self.encoder(obs).detach()
            qs = self.q_net(zs)
            action = qs[0].argmax().data.item()
        else:
            action = random.randint(0, self.N_action-1)

        return action

def get_decay(epi_iter):
    decay = math.pow(0.999, epi_iter)
    if decay < 0.05:
        decay = 0.05
    return decay

In [ ]:
""" Replace CURL encoder with VGG11 encoder head """
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)

Using cache found in /home/alexdloia/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /home/alexdloia/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
z_dim = resnet.fc.in_features
for param in resnet.parameters():
    param.requires_grad = False
resnet.fc = nn.Flatten()

In [ ]:
def main():
    env = gym.make('ALE/SpaceInvaders-v5')
    max_epi_iter = 2000
    max_MC_iter = 200
    obs = env.render(mode='rgb_array')
    agent = Agent(act_dim=env.action_space.n, max_epi_num=1000, max_epi_len=max_MC_iter, encoder=resnet, z_dim=z_dim)
    train_curve = []
    for epi_iter in range(max_epi_iter):
        random.seed()
        env.reset()
        obs = env.render(mode='rgb_array')
        returns = 0.0
        for MC_iter in range(max_MC_iter):
            # Make sure to set this to an actual action
            # TODO: decide whether to normalize pixels by / 255 (already done in PixelEncoder atm)
            action = agent.get_action(obs, get_decay(epi_iter))
            next_obs, reward, done = take_action(env, action)
            returns += reward * agent.gamma ** (MC_iter)
            agent.remember(obs, action, reward, next_obs)
            obs = next_obs.copy()
            if done or MC_iter >= max_MC_iter-1:
                agent.buffer.create_new_epi()
                break
        print('Episode', epi_iter, 'returns', returns)
        if epi_iter % 1 == 0:
            train_curve.append(returns)
        if agent.buffer.is_available():
            agent.train()
        
    print(train_curve)

In [ ]:
main()

Episode 0 returns 17.965310734569563
Episode 1 returns 45.05415062598091
Episode 2 returns 39.255811312366646
Episode 3 returns 13.057570981367652
Episode 4 returns 45.08540473773506
Episode 5 returns 93.4018575435289
Episode 6 returns 31.294492757279958
Episode 7 returns 50.03331890453741
Episode 8 returns 36.06029550371815
Episode 9 returns 68.28918148070666
Episode 10 returns 49.17981748894563
Episode 11 returns 91.93847411509236
Episode 12 returns 46.03738456036524
Episode 13 returns 54.054188643259735
Episode 14 returns 12.957189098269525
Episode 15 returns 26.90154675075939
Episode 16 returns 35.79014842177508
Episode 17 returns 17.984208691342175
Episode 18 returns 17.96618056691135
Episode 19 returns 27.47159002787088
Episode 20 returns 35.490209645413216
Episode 21 returns 29.940255181978188
Episode 22 returns 26.13779539755317
Episode 23 returns 90.31015572675268
Episode 24 returns 18.21325031699766
Episode 25 returns 9.253414235575319
Episode 26 returns 71.45863321827557
Epi

In [ ]:
for m in model.modules():
    print(m)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

Code references for DQN:

https://github.com/taochenshh/dqn-pytorch

https://github.com/transedward/pytorch-dqn (for sampling from replay buffer)

CURL code: https://github.com/MishaLaskin/curl

Code reference for using a pretrained network on imagenet (VGG):
https://pytorch.org/hub/pytorch_vision_vgg/